In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections

import nltk
from sklearn.metrics import f1_score
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import BorderlineSMOTE
from sklearn import svm

from scipy import stats
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from sklearn.pipeline import Pipeline

from collections import Counter
from nltk.data import load
from collections import deque
from scipy.sparse import hstack

%matplotlib inline

In [2]:
combined = pd.read_csv('cleanAndGenerateWordBag_v0.2/wordBag.csv', sep = '\t')
data = combined.drop_duplicates(['lyrics'])

In [3]:
artist_10 = data.artist.value_counts()[:10]
data_10 = data[data.artist.isin(artist_10.index)]

In [4]:
train, test = train_test_split(data_10, test_size=0.2, random_state = 124, stratify = data_10.artist)
corpus = train['wordBag'].values.astype(str).tolist()
corpus_test = test['wordBag'].values.astype(str).tolist()

In [5]:
svm_extrafeatures = Pipeline([
    ('tfidf', TfidfVectorizer(max_features = 3000, ngram_range = (1,1))),
    ('model', svm.SVC(gamma="scale", kernel = 'rbf',C = 20))
])

In [18]:
vectorizer = TfidfVectorizer(max_features = 3000, ngram_range = (1,1), lowercase=True, analyzer='word', norm='l2')
svm_clf = svm.SVC(gamma="scale", kernel = 'rbf',C = 20)

corpus = train['wordBag'].values.astype(str).tolist()
corpus_test = test['wordBag'].values.astype(str).tolist()

X_t = vectorizer.fit_transform(corpus)
X_test_t = vectorizer.fit_transform(corpus_test)

y = train.artist
y_true = test.artist

In [38]:
y_pred

array(['dolly-parton', 'eddy-arnold', 'eddy-arnold', ..., 'dolly-parton',
       'elton-john', 'dean-martin'], dtype=object)

In [59]:
Xtemp = vectorizer.fit_transform(corpus)
XtempTest = vectorizer.fit(corpus_test)

svm_clf.fit(Xtemp, y)
y_pred_temp = svm_clf.predict(XtempTest)
f1_score(y_true, y_pred_temp, average='macro')

TypeError: float() argument must be a string or a number, not 'TfidfVectorizer'

In [51]:
Xtemp

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=3000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [39]:
svm_clf.fit(X_t, y)
y_pred1 = svm_clf.predict(X_test_t)
f1_score(y_true, y_pred1, average = 'macro') 

0.08944655911172851

In [55]:
svm_extrafeatures.fit(corpus,y)
y_pred2 = svm_extrafeatures.predict(corpus_test)
f1_score(y_true, y_pred2, average = 'macro') 

0.4994807363928165

In [46]:
collections.Counter(y_pred1)

Counter({'dolly-parton': 196,
         'eddy-arnold': 333,
         'dean-martin': 198,
         'eminem': 7,
         'barbra-streisand': 31,
         'bob-dylan': 72,
         'elton-john': 175,
         'american-idol': 34,
         'bee-gees': 32,
         'chris-brown': 6})

In [47]:
collections.Counter(y_pred2)

Counter({'eminem': 78,
         'bee-gees': 70,
         'chris-brown': 91,
         'bob-dylan': 109,
         'barbra-streisand': 129,
         'american-idol': 132,
         'elton-john': 125,
         'eddy-arnold': 120,
         'dean-martin': 80,
         'dolly-parton': 150})

In [9]:
svm_tif = Pipeline([
    ('tfidf', TfidfVectorizer(max_features = 3000, ngram_range = (1,1))),
])

In [30]:
svm_extrafeatures.steps[1][1]

SVC(C=20, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [31]:
svm_clf.get_params

<bound method BaseEstimator.get_params of SVC(C=20, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)>

In [37]:
"analyzer='word', binary=False, decode_error='strict', \
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',\
        lowercase=True, max_df=1.0, max_features=3000, min_df=1,\
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,\
        stop_words=None, strip_accents=None, sublinear_tf=False,\
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,\
        vocabulary=None)" == "analyzer='word', binary=False, decode_error='strict',\
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',\
        lowercase=True, max_df=1.0, max_features=3000, min_df=1,\
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,\
        stop_words=None, strip_accents=None, sublinear_tf=False,\
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,\
        vocabulary=None"

False